# Semana 03 - Machine Learning

## Verificar se a variável target está balanceada

In [105]:
import pandas as pd

dados = pd.read_csv('../dados/dados_clientes_alura_voz.csv')

In [106]:
dados.churn.value_counts(normalize=True) * 100

0    73.463013
1    26.536987
Name: churn, dtype: float64

As classes estão desbalanceadas. Portanto serão aplicadas técnicas de balanceamento.

## Enconding as variáveis

In [107]:
from sklearn.preprocessing import OrdinalEncoder

colunas_encoder = ['servico_internet', 'contrato', 'metodo_pagamento']
encoder = OrdinalEncoder()
dados[colunas_encoder] = encoder.fit_transform(dados[colunas_encoder])

## Seleção variáveis

Na análise exploratória realizada na semana anterior foram verificadas que algumas variáveis não trazem informações úteis para a solução do problema.

* ID_cliente - é um identificador único de identificação do usuário
* gastos_totais - esta variável é dependente do tempo que o cliente fica na empresa, obviamente quanto maior o tempo na empresa maiores os gastos totais
* gastos_diarias - está altamente correlacionada com a variável gastos_totais já que foi derivada a partir da mesma
* genero - Em ambos os grupos, com e sem churn, a proporção de homens e mulheres é de 50% aproximadamente
* servico_telefone - Não há diferença na proporção de usuários que utilizam este serviço dos dois grupos, com e sem churn

Totas as variáveis acima serão removidas da base de dados para simplificação do problema.

Também foram verificadas os seguintes padrões:

* A proporção dos clientes que contratam serviço de backup online é próxima dos que contratam proteção para dispositivo nos dois grupos
* A proporção dos clientes que contratam serviço de streaming de TV é próxima dos que contratam streaming de filmes

Considerando-se tais padrões serão criadas duas novas variáveis:

* protecao - Representa se o cliente contrata algum serviço de proteção (backup ou proteção de dispositivo)
* streaming - Representa se o cliente contrata algum serviço de streaming (TV ou filmes)

Após a criação destas duas novas variáveis as variáveis originas (backup_online, protecao_dispositivo, streaming_TV e streaming_filmes) serão removidas.

In [108]:
# Separação dos conjuntos X e y e remoção de variáveis irrelavantes
X = dados.drop(['ID_cliente', 'churn', 'gastos_totais', 'gastos_diarios', 'genero', 'servico_telefone'], axis=1)
y = dados['churn']

# Criação da variável streaming
f_streming = lambda x: 1 if x.streaming_TV or x.streaming_filmes else 0
X['streaming'] = X.apply(f_streming, axis=1)

# Criação da variável seguranca
f_seguranca = lambda x: 1 if x.backup_online or x.protecao_dispositivo else 0
X['seguranca'] = X.apply(f_seguranca, axis=1)

# Remoção das variáveis redundantes
X.drop(['backup_online', 'protecao_dispositivo', 'streaming_TV', 'streaming_filmes'], axis=1, inplace=True)

## Balanceamento utilizando Under-Sampling

In [109]:
from imblearn.under_sampling import RandomUnderSampler

balanceador = RandomUnderSampler(random_state=101)

X_resampled, y_resampled = balanceador.fit_resample(X, y)

## Testando modelos

Serão testados os modelos: Random Forest, AdaBoost, Regressão Logística, Naive Bayes, KNN e Suport Vector Machine

In [114]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=101)
scaler = MinMaxScaler()

# Função para rodar os modelos
def run_modelo(modelo, X, y, cv):
    X = scaler.fit_transform(X)
    cv_results = cross_validate(modelo, X, y, cv=cv, n_jobs=-1, scoring=['precision', 'recall', 'accuracy'])
    acuracia_media = round(cv_results['test_accuracy'].mean(), 3)
    precisao_media =  round(cv_results['test_precision'].mean(), 3)
    recall_medio = round(cv_results['test_recall'].mean(), 3)
    return [acuracia_media, precisao_media, recall_medio]

In [115]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

rfc = RandomForestClassifier(random_state=101, n_jobs=-1)
adc = AdaBoostClassifier(random_state=101)
lr = LogisticRegression(random_state=101)
gnb = GaussianNB()
knn = KNeighborsClassifier(n_jobs=-1)
svc = SVC(random_state=101)

modelos = [rfc, adc, lr, gnb, knn, svc]
nome_modelos = ['RandomForest', 'AdaBoost', 'LinearRegression', 'NaiveBayes', 'KNN', 'SVM']
resultados = []

for i, modelo in enumerate(modelos):
    print(nome_modelos[i])
    resultados.append(run_modelo(modelo, X_resampled, y_resampled, skf))

resultados = pd.DataFrame(resultados, index=nome_modelos, columns=['Acurácia', 'Precisão', 'Recall'])
resultados

RandomForest
AdaBoost
LinearRegression
NaiveBayes
KNN
SVM


,Acurácia,Precisão,Recall
RandomForest,0.746,0.745,0.749
AdaBoost,0.766,0.743,0.813
LinearRegression,0.747,0.742,0.758
NaiveBayes,0.756,0.746,0.776
KNN,0.715,0.698,0.759
SVM,0.750,0.739,0.775


In [118]:
resultados.sort_values(by=['Acurácia', 'Recall', 'Precisão'], ascending=False)

,Acurácia,Precisão,Recall
AdaBoost,0.766,0.743,0.813
NaiveBayes,0.756,0.746,0.776
SVM,0.750,0.739,0.775
LinearRegression,0.747,0.742,0.758
RandomForest,0.746,0.745,0.749
KNN,0.715,0.698,0.759


O Melhor modelo é portanto o: AdaBoost

## Otimizando o modelo

In [129]:
from sklearn.model_selection import GridSearchCV

parametros = {
    'n_estimators': [50, 100, 150, 200, 250],
    'learning_rate': [.25, .5, .75, 1., 1.25],
}

busca_parametros = GridSearchCV(
    estimator = adc,
    param_grid = parametros,
    cv = skf
)

busca_parametros.fit(X_resampled, y_resampled)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=101, shuffle=True),
             estimator=AdaBoostClassifier(random_state=101),
             param_grid={'learning_rate': [0.25, 0.5, 0.75, 1.0, 1.25],
                         'n_estimators': [50, 100, 150, 200, 250]})

In [132]:
busca_parametros.best_estimator_

AdaBoostClassifier(learning_rate=0.25, n_estimators=150, random_state=101)

In [133]:
adc_best = AdaBoostClassifier(learning_rate=0.25, n_estimators=150, random_state=101)
print('Acurácia | Precisão | Recall')
print(run_modelo(adc_best, X_resampled, y_resampled, skf))

Acurácia | Precisão | Recall
[0.766, 0.742, 0.816]


In [134]:
resultados.loc['AdaBoost']

Acurácia    0.766
Precisão    0.743
Recall      0.813
Name: AdaBoost, dtype: float64

Devido a limitações de hardware não foi possível testar mais configurações para otimizar o modelo. Porém vê-se que houve uma pequena melhora no valor de recall.

## Teste de outras configurações de balanceamento.

### Modelo sem balanceamento

In [131]:
run_modelo(adc_best, X, y, skf)

[0.803, 0.665, 0.52]

Neste caso a alta acurácia é deve-se apenas ao fato do desbalanceamento do modelo.

## Utilizando over sampling

In [135]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=101)
X_oversampling, y_oversampling = ros.fit_resample(X, y)
run_modelo(adc_best, X_oversampling, y_oversampling, skf)

[0.767, 0.743, 0.818]

Utilizando o a técnica de oversampling obte-ve resultado semelhante ao caso de undersampling.

## Conclusões

A análise exploratória dos dados permitiu a redução de dimensionalidade e consequente a complexidade do mmodelo. Utilizando-se a técnica de undersampling consegui equilibrar as classes, assim diminuiu-se a probabilidade do modelo tornar-se tendencioso para uma determinada classe. Todos os modelos testador obtiveram resultados próximos de acurácia, apenas o KNN ficou mais distante dos demais. O modelo de AdaBoost destacouse por ser o de melhor desempenho tanto em recall (0,813) e Acurácia (0,766). Após aplicar-se técnicas de otimização pode-se obter um pequeno incremento no recall do AdaBoost (0,816). Talvez possa-se ter um incremento maior na performance do modelo mas limitações de hardware não permitiram testar esta hípotese. Ao utilizar os dados sem realizar técnica de balanceamento a acurácia foi bem elevada (0,803), porém, este resultado alto deve ao próprio desbalanceamento dos dados que faz o modelo tender a jogar mais valores para a classe dominante. Com a técnica de OverSampling, neste caso, foram obtidos resultados próximos aos do uso de Undersampling. Contudo, a técncia de oversampling deve ser utilizada com cuidado pois, quando o desbalanceamento entre as classes é muito grande, a quantidade de valores sintéticos pode comprometer as análises.